Os dados fornecidos pelo Spotify são em formato JSON, e nem todos os arquivos seguem a mesma estrutura de hierarquia.
Além disso, por mais que eu tentei reproduzir uma estrutura de raw/standard/cleaned/curated, não houve necessidade de tantas camadas no ETL, de modo que apenas raw -> stanrdard e standard -> curated foram suficientes.

Mais informações sobre o projeto: https://bit.ly/3g8Yw7b

# Raw to Standard

In [1]:
import pandas as pd
from datetime import date

In [2]:
spotify_path = "my_spotify_data/MyData/"

## Reading Files

### Streaming

In [3]:
streaming_history1= pd.read_json(spotify_path + "StreamingHistory0.json")
streaming_history2= pd.read_json(spotify_path + "StreamingHistory1.json")
streaming_history_raw = pd.concat([streaming_history1, streaming_history2])

In [4]:
streaming_history_raw.dtypes

endTime       object
artistName    object
trackName     object
msPlayed       int64
dtype: object

In [6]:
streaming_history_raw.head()

,endTime,artistName,trackName,msPlayed
0,2021-12-10 11:46,FRND,Maliboo,136999
1,2021-12-10 12:00,AJR,The Good Part,208438
2,2021-12-11 12:17,Boogarins,Foimal,7799
3,2021-12-11 12:20,Imagine Dragons,Tokyo,164327
4,2021-12-11 12:23,Jão,Você Vai Me Destruir,184692


### Library

In [7]:
my_library = pd.read_json(spotify_path + "YourLibrary.json", orient="index")

In [8]:
my_library.head()

,0,1,2,3,4,5,6,7,8,9,...,331,332,333,334,335,336,337,338,339,340
tracks,"{'artist': 'Queens of the Stone Age', 'album':...","{'artist': 'Sawyer Fredericks', 'album': 'A Go...","{'artist': '3030', 'album': 'O Outro Lado', 't...","{'artist': '3030', 'album': 'TROPICALIA', 'tra...","{'artist': 'Madeon', 'album': 'Adventure (Delu...","{'artist': 'Madeon', 'album': 'Adventure (Delu...","{'artist': 'Forfun', 'album': 'Alegria Compart...","{'artist': 'Twenty One Pilots', 'album': 'Tren...","{'artist': 'CPM 22', 'album': 'CPM22 - 20 Anos...","{'artist': 'CPM 22', 'album': 'CPM 22 - Acústi...",...,"{'artist': 'Blackway', 'album': 'Spider-Man: I...","{'artist': 'The Royal Concept', 'album': 'Roya...","{'artist': 'Madeon', 'album': 'Adventure (Delu...","{'artist': 'Aminé', 'album': 'Spider-Man: Into...","{'artist': 'Thirty Seconds To Mars', 'album': ...","{'artist': 'CPM 22', 'album': 'CPM22 - 20 Anos...","{'artist': 'Madeon', 'album': 'Adventure (Delu...","{'artist': 'Queens of the Stone Age', 'album':...","{'artist': 'Flume', 'album': 'Skin', 'track': ...","{'artist': 'GHOSTT', 'album': 'Lembrete', 'tra..."
albums,"{'artist': 'Queens of the Stone Age', 'album':...","{'artist': 'Baco Exu do Blues', 'album': 'Blue...","{'artist': 'Various Artists', 'album': 'Spider...","{'artist': '3030', 'album': 'Alquimia', 'uri':...","{'artist': 'Imagine Dragons', 'album': 'Night ...","{'artist': 'Don L', 'album': 'Roteiro pra Aïno...","{'artist': 'Twenty One Pilots', 'album': 'Tren...","{'artist': 'Don L', 'album': 'Roteiro pra Aïno...","{'artist': 'Baco Exu do Blues', 'album': 'QVVJ...","{'artist': '3030', 'album': 'TROPICALIA', 'uri...",...,None,None,None,None,None,None,None,None,None,None
shows,"{'name': 'AntiCast', 'publisher': 'Half Deaf',...","{'name': 'AntiCast', 'publisher': 'HD1', 'uri'...","{'name': 'Duolingo French Podcast', 'publisher...","{'name': 'Inglês Nu E Cru Rádio', 'publisher':...","{'name': 'Learn English Through Listening', 'p...",{'name': 'Plain English: Learn English on Spot...,{'name': 'Speak English Now with teacher Georg...,None,None,None,...,None,None,None,None,None,None,None,None,None,None
episodes,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
bannedTracks,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


#### Library Tracks

In [9]:
my_tracks = my_library.loc['tracks']
my_tracks = my_tracks.dropna()
tracks = pd.DataFrame.from_records(
    my_tracks.values, index=my_tracks.index
)

In [10]:
tracks = tracks.rename(columns = {
    "artist":"ARTIST_NAME",
    "album":"ALBUM_NAME",
    "track":"TRACK_NAME",
    "uri":"URL"
})

In [11]:
tracks.head()

,ARTIST_NAME,ALBUM_NAME,TRACK_NAME,URL
0,Queens of the Stone Age,...Like Clockwork,I Appear Missing,spotify:track:6mfKEPTYiBAYZ9z0429jsp
1,Sawyer Fredericks,A Good Storm,What I've Done,spotify:track:07xjXzCNaV0MmOKqtwjmzQ
2,3030,O Outro Lado,O Outro Lado,spotify:track:5AJikzqEavWOdBnLl5c6DP
3,3030,TROPICALIA,GATO PRETO,spotify:track:1bRCBPPSeAKhxL2UXGvfpW
4,Madeon,Adventure (Deluxe),Technicolor,spotify:track:3OgU43E7rPo3u9fNEIpdWi


#### Library Albumns

In [12]:
my_albums = my_library.loc['albums']
my_albums = my_albums.dropna()
albums = pd.DataFrame.from_records(
    my_albums.values, index=my_albums.index
)

In [13]:
albums = albums.rename(columns = {
    "artist":"ARTIST_NAME",
    "album":"ALBUM_NAME",
    "uri":"URL"
})

In [14]:
albums.head()

,ARTIST_NAME,ALBUM_NAME,URL
0,Queens of the Stone Age,...Like Clockwork,spotify:album:06S2JBsr4U1Dz3YaenPdVq
1,Baco Exu do Blues,Bluesman,spotify:album:0QMVSKhzT4u2DEd8qdlz4I
2,Various Artists,Spider-Man: Into the Spider-Verse,spotify:album:3pOj4mmxL5MUwR1IR8LPzL
3,3030,Alquimia,spotify:album:0oliR7LznCi1Bio9mTBeQZ
4,Imagine Dragons,Night Visions,spotify:album:6nxDQi0FeEwccEPJeNySoS


## Saving to Parquet

In [15]:
def save_to_parquet(df, parquet_name):
    today = date.today()
    df['EXPORT_DAY'] = today.day
    df['EXPORT_MONTH'] = today.month
    df['EXPORT_YEAR'] = today.year
    df.to_parquet("standard/" + parquet_name, partition_cols = ['EXPORT_YEAR', 'EXPORT_MONTH', 'EXPORT_DAY'])

In [16]:
save_to_parquet(tracks, "tracks_standard")

In [17]:
save_to_parquet(streaming_history_raw, "streaming_history_standard")

In [18]:
save_to_parquet(albums, "albumns_standard")